In [7]:
%load_ext autoreload
%autoreload 2
import os
import sys
import yaml
import warnings
import numpy as np

cwd = '/teamspace/studios/this_studio/letsdoit'
if (cwd not in sys.path):
    sys.path.append(cwd)
sys.path.append(os.path.dirname(cwd))

# from utils.misc import plot_point_cloud_and_object
# from scipy.spatial import KDTree
from pipeline.pipeline import retrieve_best_action_object

from pipeline.pipeline import Pipeline
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
path_config = '/teamspace/studios/this_studio/letsdoit/config/config_test.yml'

In [4]:
with open(path_config, 'r') as file:
    cfg = yaml.safe_load(file)

In [5]:
pipe = Pipeline(**cfg)

Initializing ClipRetriver...
Initializing the MasksFinder...
final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>
Initializing MasksMerger...
Initializing ClipModel...
Initializing ClipProcessor...
Done!


In [ ]:
# action_objects = pipe.run()

In [56]:
# Debug
visit_id = '421380'
instruction_block = [ib for ib in pipe.instruction_list if str(ib['visit_id'])==visit_id][0]
visit_id = str(instruction_block['visit_id'])
# desc_id = 'a10e8718-1997-421c-9a67-7fa25deb7200'
# ins = [i for i in instruction_block['instructions'] if i['desc_id']==desc_id][0]
visit_id = str(instruction_block['visit_id'])

# load the point cloud
pipe.pcd = pipe.loader.load_pcd(visit_id)
# load image_paths, poses, depth_paths, etc.
dict_data = pipe._load_data(visit_id)
# load image_features
image_features_path = pipe.loader.get_image_features_path(visit_id)
pipe.retriever.update_image_features(image_features_path) 

object_labels = instruction_block['all_objects']
instructions = instruction_block['instructions']

objects = []
for object_label in object_labels:
    print(f"Extracting objects for '{object_label}'")
    objs = pipe._get_objects_3d(object_label, dict_data)
    if len(objs) > 0:
        objects.extend(objs)



Loading rgb frames from visit 421380 and video 42445021: 100%|██████████| 139/139 [00:00<00:00, 62094.82it/s]
Loading rgb frames from visit 421380 and video 42445022: 100%|██████████| 306/306 [00:00<00:00, 85895.93it/s]
Loading rgb frames from visit 421380 and video 42445026: 100%|██████████| 209/209 [00:00<00:00, 87960.02it/s]
Loading depth frames from visit 421380 and video 42445021: 100%|██████████| 139/139 [00:00<00:00, 82450.61it/s]
Loading depth frames from visit 421380 and video 42445022: 100%|██████████| 306/306 [00:00<00:00, 50900.54it/s]
Loading depth frames from visit 421380 and video 42445026: 100%|██████████| 209/209 [00:00<00:00, 47028.41it/s]


Extracting objects for 'knob'


Denoising point clouds: 100%|██████████| 17/17 [00:07<00:00,  2.21it/s]


Extracting objects for 'drawer'


Denoising point clouds: 100%|██████████| 20/20 [00:06<00:00,  3.30it/s]


...

Denoising point clouds: 100%|██████████| 13/13 [00:05<00:00,  2.53it/s]


Extracting objects for 'cabinet'


Denoising point clouds: 100%|██████████| 10/10 [00:02<00:00,  3.44it/s]


Extracting objects for 'ceiling'


Denoising point clouds: 100%|██████████| 16/16 [00:02<00:00,  6.60it/s]


In [57]:
action_objects = []
for instruction in instructions:
    try:
        action_object = retrieve_best_action_object(instruction, objects)
    except:
        action_object = None

    if pipe.path_submission_folder is not None:
        pipe._save_instruction_result(visit_id, instruction, [action_object])
    action_objects.append(action_object)